In [1]:
import torch
import pandas as pd
from ultralytics import YOLO
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import preprocess_image, show_cam_on_image
import cv2
import numpy as np
import os

In [ ]:
# i not sure which one is your model import yourself
model = YOLO('path/to/yolov8/model.pt')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

target_layer = model.model.model[-1]

image_paths = [r"C:\Users\dunli\Documents\STSY-project-main\Training Data\test\images"] # need to change to your testing image

results_df = pd.DataFrame(columns=['Image', 'Class', 'Confidence', 'GradCAM_Path'])

for image_path in image_paths:
    rgb_img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    rgb_img = np.float32(rgb_img) / 255

    input_tensor = preprocess_image(
        rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
    ).to('cuda' if torch.cuda.is_available() else 'cpu')

    cam = GradCAMPlusPlus(model=model.model, target_layers=[target_layer])
    grayscale_cam = cam(input_tensor=input_tensor)[0]

    heatmap = cv2.applyColorMap(
        np.uint8(255 * grayscale_cam), cv2.COLORMAP_JET
    )
    heatmap = heatmap.astype(np.float32) / 255

    alpha = 0.3
    final_output = cv2.addWeighted(rgb_img, alpha, heatmap, 1 - alpha, 0)

    heatmap_path = f"gradcam_{os.path.basename(image_path)}"
    cv2.imwrite(heatmap_path, (final_output * 255).astype(np.uint8))

    yolo_results = model(image_path)
    for result in yolo_results:
        for box in result.boxes:
            class_name = box.cls
            confidence = box.conf

            results_df = results_df.append({
                'Image': image_path,
                'Class': class_name,
                'Confidence': confidence,
                'GradCAM_Path': heatmap_path
            }, ignore_index=True)

print(results_df)